# Data Mining Portfolio

## Project Overview
Brief description of the business problem and objective.

## Stage 1: Problem Definition and Data Scoping
### Business Problem
Customer churn is a significant challenge for telecom service providers, as acquiring new customers is typically more costly than retaining existing ones. UK telecom operators such as Three UK, EE, and Vodafone operate in a highly competitive market where customers can switch providers easily due to short-term contracts and similar pricing structures. As a result, the ability to identify customers who are at risk of leaving is critical for improving customer retention and protecting recurring revenue.

### Data Mining Objective
The objective of this project is to develop a predictive model that can identify customers who are likely to churn based on their historical service usage, contract details, and billing information. This is framed as a binary classification problem, where the output variable indicates whether a customer churns or remains with the service.

### Dataset Selection and Justification
This study uses a publicly available telecom customer churn dataset sourced from Kaggle, originally provided by IBM. The dataset contains detailed customer-level information including tenure, contract type, monthly charges, payment method, subscribed services, and a churn indicator. Although the dataset does not originate from a specific UK provider, it closely reflects the characteristics of customers in the UK telecom sector and is widely used for churn prediction research. The dataset is therefore appropriate for demonstrating data mining techniques and evaluating predictive performance, while acknowledging that real-world deployment would require operator-specific data.

## Stage 2: Exploratory Data Analysis and Pre-processing
- Data overview
- Missing values and outliers
- Visualisations
- Feature engineering

## Stage 3: Data Mining / Machine Learning
- Technique(s) used
- Justification
- Model training and validation

## Stage 4: Evaluation, Recommendations, and Reflection
- Model evaluation
- Business implications
- Limitations and future work
- Ethical, privacy, and security considerations

## References
(UWE Harvard format)